In [13]:
# 챗봇, 번역, 이미지 캡셔닝등에 사용되는 시퀀스 학습/생성 모델인 Seq2Seq 을 구현해봅니다.
# 영어 단어를 한국어 단어로 번역하는 프로그램을 만들어봅니다.
import josa_Remove
import tensorflow as tf
import numpy as np

# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd']
#       to   -> ['t', 'o', 'P', 'P']

num_dic, r_bow = josa_Remove.total()
print(num_dic)
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터



{'우리는': 0, '번이고': 1, '대화란': 2, '모습이기': 3, '줄': 4, '나로': 5, '그때마다': 6, '손바닥으로': 7, '밤바다의': 8, '사람들': 9, '이제': 10, '그리울': 11, '인숙이라고요인숙이': 12, '빈': 13, '마신': 14, '달리고': 15, '자넨': 16, '어머니의': 17, '시찰원': 18, '들렀던': 19, '말했다아이': 20, '좋은': 21, '죽겠어요여자는': 22, '얘기들을': 23, '서울보다는': 24, '성악을': 25, '강의가': 26, '아니에요여자는': 27, '못한': 28, '매일': 29, '틈으로': 30, '나일론의': 31, '강하게': 32, '통지가': 33, '반수면': 34, '싹': 35, '어디선지': 36, '되실': 37, '킬로': 38, '무진에의': 39, '무책임도': 40, '늦은': 41, '은': 42, '비틀거릴': 43, '스피커에서는': 44, '어두컴컴했다': 45, '밑에': 46, '급히': 47, '고뇌의': 48, '현실과': 49, '원하지만': 50, '하다': 51, '없네': 52, '사라져갔다내': 53, '자넨전': 54, '꾹': 55, '지향형인': 56, '박이': 57, '심부름을': 58, '그려': 59, '조를': 60, '이마를': 61, '가슴이': 62, '실감했다이번에': 63, '출신': 64, '계세요여자가': 65, '좋아하고': 66, '어디서': 67, '머리는': 68, '맡고': 69, '전보와': 70, '나윤희중': 71, '여러': 72, '아니겠죠옛날의': 73, '살고': 74, '그것은': 75, '덜하는': 76, '운수를': 77, '붓게': 78, '남자에게': 79, '잊고': 80, '오는': 81, '믿음을': 82, '가': 83, '둘러쌌고': 84, '제약회사의': 85, '과거의': 86, '꼽을': 87, '웃었다내': 88, '살기로': 

In [14]:
import csv
 
seq_data = []
f = open('sample1.csv', 'r')
rdr = csv.reader(f)
for line in rdr:
    seq_data.append(line)
    #print(line)
    
print(seq_data)



[['synopsis', 'phrase'], ['공항에서 수하물 노동자로 일하며 음악의 꿈을 키우던 이민자 출신의 아웃사이더 ‘파록버사라’ \n 보컬을 구하던 로컬 밴드에 들어가게 되면서 ‘프레디 머큐리’라는 이름으로 밴드 ‘퀸’을 이끌게 된다. \n  \n 시대를 앞서가는 독창적인 음악과 화려한 퍼포먼스로 관중들을 사로잡으며 성장하던 ‘퀸’은 \n 라디오와 방송에서 외면을 받을 것이라는 음반사의 반대에도 불구하고 \n 무려 6분 동안 이어지는 실험적인 곡 ‘보헤미안 랩소디’로 대성공을 거두며 월드스타 반열에 오른다. \n  \n 그러나 독보적인 존재감을 뿜어내던 ‘프레디 머큐리’는 솔로 데뷔라는 유혹에 흔들리게 되고 \n 결국 오랜 시간 함께 해왔던 멤버들과 결별을 선언하게 되는데… \n  \n 세상에서 소외된 아웃사이더에서 전설의 록밴드 ‘퀸’이 되기까지, \n 우리가 몰랐던 그들의 진짜 이야기가 시작된다!', '“나는 스타가 되지 않을 것이다, 전설이 될 것이다”'], ['‘뉴트 스캐맨더’(에디 레드메인)의 활약으로 \n 강력한 어둠의 마법사 ‘겔러트 그린델왈드’(조니 뎁)가 미합중국 마법부 MACUSA에 붙잡히지만, \n 이내 장담했던 대로 탈출해 추종자를 모으기 시작한다. \n 순혈 마법사의 세력을 모아 마법을 사용하지 않는 사람들을 지배하려는 \n 그린델왈드의 야욕을 막기 위해 ‘알버스 덤블도어’(주드 로)는 제자였던 뉴트에게 도움을 요청한다. \n 마법사 사회는 점점 더 분열되어 가는 가운데, \n 앞날의 위험을 알지 못한 채 뉴트는 이를 승낙하는데…', '마법 세계의 운명을 건 대결이 시작된다!'], ['1997년, 대한민국 최고의 경제 호황을 믿어 의심치 않았던 그때, \n 곧 엄청난 경제 위기가 닥칠 것을 예견한 한국은행 통화정책팀장 ‘한시현’(김혜수)은 이 사실을 보고하고, \n 정부는 뒤늦게 국가부도 사태를 막기 위한 비공개 대책팀을 꾸린다. \n  \n 한편, 곳곳에서 감지되는 위기의 시그널을 포착하고 과감히 사표를 던진 금융맨 ‘윤정학

In [15]:
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만든다.
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여준다.
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙인다.
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch


#########
# 옵션 설정
######
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
# 입력과 출력의 형태가 one-hot 인코딩으로 같으므로 크기도 같다.
n_class = n_input = dic_len


#########
# 신경망 모델 구성
######
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다.
# [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
# [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])


# 인코더 셀을 구성한다.
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)

# 디코더 셀을 구성한다.
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을
    # 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심.
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)


model = tf.layers.dense(outputs, n_class, activation=None)


cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)



ValueError: Variable encode/rnn/basic_rnn_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 1718, in __init__
    self._traceback = self._graph._extract_stack()  # pylint: disable=protected-access
  File "C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\ops.py", line 3392, in create_op
    op_def=op_def)
  File "C:\Users\User\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 787, in _apply_op_helper
    op_def=op_def)


In [16]:

#########
# 신경망 모델 학습
######
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),
          'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')


#########
# 번역 테스트
######
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채운다.
    # ['word', 'PPPP']
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만든다.
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다.
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다.
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated


print('\n=== 번역 테스트 ===')

print('word ->', translate('그리스도가 부활한 지 약 30년 후인 AD 67년, 로마제국의 황제 ‘네로’는 자신의 광기로 일어난 대화재의 원인으로 신흥종교인 기독교에 책임을 돌린다. 짐승들의 먹이가 되거나 불태워지며 극심한 박해를 받는 크리스천들.'))


KeyError: 's'